In [1]:
import numpy as np
import pandas as pd

In [ ]:
Book=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ReccomenderSystem/BookReccomendation/Books.csv")
Rating=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ReccomenderSystem/BookReccomendation/Ratings.csv")
User=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ReccomenderSystem/BookReccomendation/Users.csv")


In [ ]:
Book.head(5)

In [ ]:
Rating.head(5)

In [ ]:
User.head(5)

In [ ]:
print(Book.shape)
print(Rating.shape)
print(User.shape)

In [ ]:
##checking null values in all three data frames
Book.isnull().sum()

In [ ]:
Rating.isnull().sum()

In [ ]:
User.isnull().sum()

# Popularity Base Recommendation

In [ ]:
# in that system we are using mean/avg to find top latest book we may use max,min,mode also
Rating_Book=Rating.merge(Book,on="ISBN",how="inner")
Rating_Book.head(2)

In [ ]:
Rating_with_num=Rating_Book.groupby('Book-Title').count()['Book-Rating'].reset_index()
Rating_with_num.rename(columns={"Book-Rating":"Total-Rating"},inplace=True)
Rating_with_num.head(5)


In [ ]:
Rating_with_avg=Rating_Book.groupby('Book-Title').mean()['Book-Rating'].reset_index()
Rating_with_avg.rename({"Book-Rating":"avg-Rating"},inplace=True)
Rating_with_avg.head(5)

In [ ]:
popular_df=Rating_with_num.merge(Rating_with_avg,on="Book-Title",how="inner")
popular_df

In [ ]:
popular_df=popular_df[popular_df['Total-Rating']>=250].sort_values('Book-Rating',ascending=False).head(50)
# return how much top popular book you want to print/show
popular_df

In [17]:
# now We Top 50 books in our popular datafram
# we need more information about selected books so we use merge operation with books details
popular_df=Book.merge(popular_df,on="Book-Title").drop_duplicates()[['Book-Title','Book-Author','Total-Rating','Book-Rating','Image-URL-M']]


KeyError: ignored

In [18]:
popular_df

,Book-Title,Book-Author,Total-Rating,Book-Rating,Image-URL-M
0,To Kill a Mockingbird,Harper Lee,510,4.700000,http://images.amazon.com/images/P/0446310786.0...
1,To Kill a Mockingbird,Harper Lee,510,4.700000,http://images.amazon.com/images/P/0446310492.0...
2,To Kill a Mockingbird,Harper Lee,510,4.700000,http://images.amazon.com/images/P/0060935464.0...
3,To Kill a Mockingbird,Harper Lee,510,4.700000,http://images.amazon.com/images/P/0397001517.0...
4,To Kill a Mockingbird,Harper Lee,510,4.700000,http://images.amazon.com/images/P/006017322X.0...
...,...,...,...,...,...
191,Hard Eight : A Stephanie Plum Novel (A Stephan...,Janet Evanovich,269,3.825279,http://images.amazon.com/images/P/1559277254.0...
192,Hard Eight : A Stephanie Plum Novel (A Stephan...,Janet Evanovich,269,3.825279,http://images.amazon.com/images/P/1559277246.0...
193,Hard Eight : A Stephanie Plum Novel (A Stephan...,Janet Evanovich,269,3.825279,http://images.amazon.com/images/P/1559277238.0...
194,Seven Up (A Stephanie Plum Novel),Janet Evanovich,278,3.888489,http://images.amazon.com/images/P/0312265840.0...


# Colaborative base filter

In [50]:
# select the user from users where each user gives atleast 200 reviews on book
x=Rating_Book.groupby("User-ID").count()['Book-Rating']>200
ValueableUsers=x[x].index
ValueableUsers


Int64Index([   254,   2276,   2766,   2977,   3363,   4017,   4385,   6251,
              6323,   6543,
            ...
            271705, 273979, 274004, 274061, 274301, 274308, 275970, 277427,
            277639, 278418],
           dtype='int64', name='User-ID', length=811)

In [ ]:
Filtered_User=Rating_Book[Rating_Book['User-ID'].isin(ValueableUsers)]
Filtered_User

In [58]:
#select the book from books where each book rate by atleast 50 times by users
# from the Selected Users
y=Filtered_User.groupby("Book-Title").count()['Book-Rating']>=50
ValueableBooks=y[y].index


Index(['1984', '1st to Die: A Novel', '2nd Chance', '4 Blondes',
       'A Bend in the Road', 'A Case of Need',
       'A Child Called \It\": One Child's Courage to Survive"',
       'A Civil Action', 'A Day Late and a Dollar Short', 'A Fine Balance',
       ...
       'Winter Solstice', 'Wish You Well', 'Without Remorse',
       'Wizard and Glass (The Dark Tower, Book 4)', 'Wuthering Heights',
       'Year of Wonders', 'You Belong To Me',
       'Zen and the Art of Motorcycle Maintenance: An Inquiry into Values',
       'Zoya', '\O\" Is for Outlaw"'],
      dtype='object', name='Book-Title', length=706)

In [ ]:
Filtered_Books=Filtered_User[Filtered_User['Book-Title'].isin(ValueableBooks)]
Filtered_Books.drop_duplicates()

In [ ]:
# Creating @2d Matrix on the basis of UserId as Column and book Tile As row
pt=Filtered_Books.pivot_table(index="Book-Title",columns="User-ID",values="Book-Rating")
pt

In [68]:
pt.fillna(0,inplace=True)
pt

User-ID,254,2276,2766,2977,3363,4017,4385,6251,6323,6543,...,271705,273979,274004,274061,274301,274308,275970,277427,277639,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Bend in the Road,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# we got all the data ppoint in 2d matrix now we need to calculate distance between them

In [63]:
from sklearn.metrics.pairwise import cosine_similarity

In [67]:
similarity_score=cosine_similarity(pt)
similarity_score

array([[1.        , 0.10255025, 0.01220856, ..., 0.12110367, 0.07347567,
        0.04316046],
       [0.10255025, 1.        , 0.2364573 , ..., 0.07446129, 0.16773875,
        0.14263397],
       [0.01220856, 0.2364573 , 1.        , ..., 0.04558758, 0.04938579,
        0.10796119],
       ...,
       [0.12110367, 0.07446129, 0.04558758, ..., 1.        , 0.07085128,
        0.0196177 ],
       [0.07347567, 0.16773875, 0.04938579, ..., 0.07085128, 1.        ,
        0.10602962],
       [0.04316046, 0.14263397, 0.10796119, ..., 0.0196177 , 0.10602962,
        1.        ]])

In [114]:
def Recommend(book_name):
  index=np.where(pt.index==book_name)[0][0]
  top_five_book=sorted(list(enumerate(similarity_score[index])),key=lambda x:x[1],reverse=True)[1:6]
  book_list=[]
  for i in top_five_book:
    book_list.append(pt.index[i[0]])
  return book_list

  

In [104]:
sorted(list(enumerate(similarity_score[0])),key=lambda x:x[1],reverse=True)[1:6]

[(47, 0.2702651417103732),
 (545, 0.2639619371123496),
 (82, 0.2366937434740099),
 (634, 0.23299389358170397),
 (551, 0.2262639743141286)]

In [115]:
Recommend("You Belong To Me")
# Rating_Book
# popular_df[popular_df['Book-Title']=='To Kill a Mockingbird']

['Loves Music, Loves to Dance',
 "I'll Be Seeing You",
 'Before I Say Good-Bye',
 "Daddy's Little Girl",
 'All Around the Town']